# **CNN**

In [ ]:
import numpy as np

In [ ]:
#extracting data from csv files into numpy arrays
from numpy import genfromtxt
x_train = genfromtxt('train_data1.csv', delimiter=',')
y_train = genfromtxt('train_labels1.csv', delimiter=',')
x_test = genfromtxt('test_data1.csv', delimiter=',')
y_test = genfromtxt('test_labels1.csv', delimiter=',')

In [ ]:
#shape
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((108, 180), (27, 180), (108,), (27,))

In [ ]:
#converting to one hot
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=9)
y_test = to_categorical(y_test, num_classes=9)
y_train.shape,y_test.shape

((108, 9), (27, 9))

In [ ]:
#reshaping to 2D 
x_train=np.reshape(x_train,(x_train.shape[0], 36,5))
x_test=np.reshape(x_test,(x_test.shape[0], 36,5))
x_train.shape,x_test.shape

((108, 36, 5), (27, 36, 5))

In [ ]:
#reshaping to shape required by CNN
x_train1=np.reshape(x_train,(x_train.shape[0], 36,5,1))
x_test1=np.reshape(x_test,(x_test.shape[0], 36,5,1))

In [ ]:
#shapes
x_train1.shape,x_test1.shape

((108, 36, 5, 1), (27, 36, 5, 1))

In [ ]:
from keras import Sequential
from keras import optimizers
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout, LSTM

In [ ]:
#adding layers and forming the model
model = Sequential()
model.add(Conv2D(64,kernel_size=5,strides=1,padding="Same",activation="relu",input_shape=(36,5,1)))
model.add(MaxPooling2D(padding="same"))

model.add(Conv2D(128,kernel_size=5,strides=1,padding="same",activation="relu"))
model.add(MaxPooling2D(padding="same"))
model.add(Dropout(0.3))


model.add(Dense(64,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(32,activation="relu"))
model.add(Dropout(0.3))

model.add(Flatten())

#model.add(Dense(9,activation="softmax"))

# **LSTM**

In [ ]:
from keras import Sequential
from keras import optimizers
from keras.layers import Dense,Dropout,Activation,LSTM,SimpleRNN, Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, Flatten, Concatenate

In [ ]:
#adding layers and forming the model
model1 = Sequential()
model1.add(LSTM(256,activation="tanh", input_shape=(36,5), return_sequences=True))
model1.add(Dropout(0.15))

model1.add(LSTM(128, activation="tanh"))
model1.add(Dropout(0.15))

model1.add(Dense(64, activation='tanh'))
model1.add(Dropout(0.15))

model1.add(Flatten())
#model1.add(Dense(units=9,activation='softmax'))

"#compiling\n\nmodel1.compile(optimizer=optimizers.Adam(lr=0.001),\n              loss='categorical_crossentropy',\n              metrics=['accuracy'])"

In [ ]:
#fitting
"history = model1.fit(x_train,y_train,epochs=3,validation_data=(x_test,y_test),batch_size=40)"

'history = model1.fit(x_train,y_train,epochs=3,validation_data=(x_test,y_test),batch_size=40)'

# **Merge**

In [ ]:
merged_model = Concatenate() ([model.output, model1.output])

x = Dense(512, activation='relu')(merged_model)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
'''x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)'''
output = Dense(9, activation= 'softmax')(x)

In [ ]:
from keras.models import Model

In [ ]:
final_model = Model(inputs=[model.input, model1.input], outputs = [output])

In [ ]:
final_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
final_model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(final_model, to_file='earlyfusionmodel.png', show_shapes=True, show_layer_names=True)

In [ ]:
history1 = final_model.fit([x_train1, x_train], y_train,batch_size=64,epochs=100,verbose=1,validation_split=0.3)

In [ ]:
#Saving the model
import h5py
model.save('CNNmodel.h5')


In [ ]:
import matplotlib.pyplot as plt


# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
#train and test loss and scores respectively
train_loss_score=final_model.evaluate(x_train,y_train)
test_loss_score=final_model.evaluate(x_test1,y_test)
print(train_loss_score)
print(test_loss_score)